In [ ]:
# Batch Inference using ray

In [ ]:
%pip install -q -U "ray[data]" transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 MB 8.8 MB/s eta 0:00:00


In [ ]:
from typing import Dict
import numpy as np
import ray

In [ ]:
ds = ray.data.from_numpy([[
    "Complete this",
    "for me",
    "This is an example sentence",
    "Ray makes distributed computing easy",
    "I am learning Ray datasets",
    "This is a test of generating multiple sentences",
    "The weather is nice today",
    "Ray can scale machine learning workloads",
    "This is another sentence to complete the array",
    "We are using Ray with HuggingFace"
]])

In [ ]:
ds.show(2)

2024-10-09 10:23:57,740	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-10-09_10-05-09_260956_935/logs/ray-data
2024-10-09 10:23:57,746	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> LimitOperator[limit=2]


Running 0: 0.00 row [00:00, ? row/s]

- limit=2 1: 0.00 row [00:00, ? row/s]

{'data': 'Complete this'}
{'data': 'for me'}


In [ ]:
class HuggingFacePredictor:
    def __init__(self):
        from transformers import pipeline
        # Initialize a pre-trained GPT2 Huggingface pipeline.
        self.model = pipeline("text-generation", model="gpt2")
        self.batch_count = 0  # Add this line

    # Logic for inference on 1 batch of data.
    def __call__(self, batch: Dict[str, np.ndarray]) -> Dict[str, list]:
        # Print the contents of the batch
        print(f"Batch contents: {batch}")

        # Get the predictions from the input batch.
        predictions = self.model(list(batch["data"]), max_length=20, num_return_sequences=1)
        # `predictions` is a list of length-one lists. For example:
        # [[{'generated_text': 'output_1'}], ..., [{'generated_text': 'output_2'}]]
        # Modify the output to get it into the following format instead:
        # ['output_1', 'output_2']
        batch["output"] = [sequences[0]["generated_text"] for sequences in predictions]

        self.batch_count += 1  # Increment the batch count
        print(f"Processed {self.batch_count} batches")  # Print the number of batches processed

        return batch
predictions = ds.map_batches(HuggingFacePredictor, concurrency=2,batch_size=5)


In [ ]:
arr=predictions.take_all()

2024-10-09 10:24:10,235	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-10-09_10-05-09_260956_935/logs/ray-data
2024-10-09 10:24:10,238	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> ActorPoolMapOperator[MapBatches(HuggingFacePredictor)]


Running 0: 0.00 row [00:00, ? row/s]

(_MapWorker pid=7769) 2024-10-09 10:24:19.772399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(_MapWorker pid=7769) 2024-10-09 10:24:19.846778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(_MapWorker pid=7769) 2024-10-09 10:24:19.891257: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(_MapWorker pid=7769) 2024-10-09 10:24:19.937284: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(_MapWorker pid=7769) To enable the following instructions: AVX2 FMA, in other operations,

- MapBatches(HuggingFacePredictor) 1: 0.00 row [00:00, ? row/s]

(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Batch contents: {'data': array(['Complete this', 'for me', 'This is an example sentence',
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768)        'Ray makes distributed computing easy',
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768)        'I am learning Ray datasets'], dtype=object)}


(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
(_MapWorker pid=7768) 2024-10-09 10:24:20.164699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(_MapWorker pid=7768) 2024-10-09 10:24:20.247430: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has alrea

(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Processed 1 batches
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Batch contents: {'data': array(['This is a test of generating multiple sentences',
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768)        'The weather is nice today',
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768)        'Ray can scale machine learning workloads',
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768)        'This is another sentence to complete the array',
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768)        'We are using Ray with HuggingFace'], dtype=object)}


(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
2024-10-09 10:24:37,878	WARNING actor_pool_map_operator.py:265 -- To ensure full parallelization across an actor pool of size 2, the Dataset should consist of at least 2 distinct blocks. Consider increasing the parallelism when creating the Dataset.


(MapWorker(MapBatches(HuggingFacePredictor)) pid=7768) Processed 2 batches


In [ ]:
arr

[{'data': 'Complete this',
  'output': 'Complete this quest every 10 seconds.\n\n25 Quest Finisher: Nilesian (3)'},
 {'data': 'for me',
  'output': "for me, if I wasn't dead, to look at her and ask if she remembers anything."},
 {'data': 'This is an example sentence',
  'output': 'This is an example sentence in which the idea of being in a high-performance sport can be a'},
 {'data': 'Ray makes distributed computing easy',
  'output': 'Ray makes distributed computing easy to understand and is available as a open source project.\n\nDocker'},
 {'data': 'I am learning Ray datasets',
  'output': 'I am learning Ray datasets from Sperc. A couple of key observations:\n\nWhile I'},
 {'data': 'This is a test of generating multiple sentences',
  'output': 'This is a test of generating multiple sentences by typing a single letter in Google. We use a number'},
 {'data': 'The weather is nice today',
  'output': "The weather is nice today and we'll stay cool in here for more than a week before we re